<a href="https://colab.research.google.com/github/WPela/DataScience_2023/blob/main/Derivatives/Option_pricing_BS_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm

### Black Scholes

In [ ]:
S_0 = 48 #initial price
K = 50 #strike price
r = 0.1  #risk
sigma = 0.20 #volatility
T = 0.25 # 3 months
N = 252 # 252 days in a year
dt = T / N # time step
N_SIMS = 1_000_000 # number of simulations
discount_factor = np.exp(-r * T)

In [ ]:
def black_scholes_analytical(S_0, K, T, r, sigma, type="call"):
    """
    Function used for calculating the price of European options using the
    analytical form of the Black-Scholes model.

    Parameters
    ------------
    s_0 : float
        Initial stock price
    K : float
        Strike price
    T : float
        Time to maturity in years
    r : float
        Annualized risk-free rate
    sigma : float
        Standard deviation of the stock returns
    type : str
        Type of the option. Can be one of the following: ["call", "put"]

    Returns
    -----------
    option_premium : float
        The premium on the option calculated using the Black-Scholes model
    """

    d1 = (
        np.log(S_0/K) + (r+0.5*sigma**2) * T) / (sigma*np.sqrt(T)
    )
    d2 = d1 - sigma * np.sqrt(T)
    if type == "call":
        N_d1 = norm.cdf(d1, 0, 1)
        N_d2 = norm.cdf(d2, 0, 1)
        val = S_0 * N_d1 - K * np.exp(-r * T) * N_d2
    else:
        type == "put"
        N_d1 = norm.cdf(-d1, 0, 1)
        N_d2 = norm.cdf(-d2, 0, 1)
        val = K * np.exp(-r * T) * N_d2 - S_0 * N_d1


    return val

Europiean call option price

In [ ]:
black_scholes_analytical(S_0=S_0, K=K, T=T,
                         r=r, sigma=sigma,
                         type="call")

1.5707193922759046

### Geometric Brownian Motion

In [ ]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N, random_seed=42,
                 antithetic_var=False):
    """
    Function used for simulating stock returns using Geometric Brownian Motion.

    Parameters
    ------------
    s_0 : float
        Initial stock price
    mu : float
        Drift coefficient
    sigma : float
        Diffusion coefficient
    n_sims : int
        Number of simulations paths
    dt : float
        Time increment, most commonly a day
    T : float
        Length of the forecast horizon, same unit as dt
    N : int
        Number of time increments in the forecast horizon
    random_seed : int
        Random seed for reproducibility
    antithetic_var : bool
        Boolean whether to use antithetic variates approach to reduce variance

    Returns
    -----------
    S_t : np.ndarray
        Matrix (size: n_sims x (T+1)) containing the simulation results.
        Rows respresent sample paths, while columns point of time.
    """

    np.random.seed(random_seed)

    # time increment
    dt = T/N

    # Brownian
    dW = np.random.normal(scale = np.sqrt(dt),
                              size=(n_sims, N + 1))

    # simulate the evolution of the process
    S_t = s_0 * np.exp(np.cumsum((mu - 0.5 * sigma ** 2) * dt + sigma * dW,
                                 axis=1))
    S_t[:, 0] = s_0

    return S_t

In [ ]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma,
                        n_sims=N_SIMS, T=T, N=N)

European call option premium simulated with Geometric Brownian Motion

In [ ]:
premium = (
    discount_factor * np.mean(np.maximum(0, gbm_sims[:, -1] - K))
)
premium

1.5710143386417244